In [1]:
import tropycal.tracks as tracks 
import datetime as dt 
import pandas as pd 

In [2]:
# Get basin data with info about all storms in all years
basin = tracks.TrackDataset(basin='both', source='hurdat', include_btk=True)

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (1.86 seconds)
--> Starting to read in best track data
--> Completed reading in best track data (9.7 seconds)


## Get All Named Storms
Return all the storms that occurred in each year

In [3]:
# Return storm data for all seasons 
storm_data = basin.to_dataframe()

# Loop through each season and get all storms
storm_details = []
for year in storm_data.index[:2]:
    season = basin.get_season(year).to_dataframe()
    season['year'] = year 
    storm_details.append(season)

# Concatenate all storms into one dataframe
all_storms = pd.concat(storm_details).set_index('year')

In [4]:
all_storms.head()

,id,name,vmax,mslp,category,ace,start_time,end_time,start_lat,start_lon
year,,,,,,,,,,
1851,AL011851,UNNAMED,80,NaN,1,4.9,1851-06-25 00:00:00,1851-06-28 00:00:00,28.0,-94.8
1851,AL021851,UNNAMED,80,NaN,1,0.6,1851-07-05 12:00:00,1851-07-05 12:00:00,22.2,-97.6
1851,AL031851,UNNAMED,50,NaN,0,0.2,1851-07-10 12:00:00,1851-07-10 12:00:00,12.0,-60.0
1851,AL041851,UNNAMED,100,NaN,3,21.8,1851-08-16 00:00:00,1851-08-27 18:00:00,13.4,-48.0
1851,AL051851,UNNAMED,50,NaN,0,4.0,1851-09-13 00:00:00,1851-09-16 18:00:00,32.5,-73.5


# Detailed Info About Each Storm
Get detailed tracking info about each storm and its path

In [5]:
# Get detailed data about individual storm
all_storms_detailed = []

year_storm_id = zip(all_storms.index, all_storms['id'])
for year, id in year_storm_id:
    storm = basin.get_storm(id).to_dataframe()
    storm['storm_id'] = id 
    storm['year'] = year
    all_storms_detailed.append(storm)

# Concatenate all the data 
all_storms_detailed = pd.concat(all_storms_detailed)

In [6]:
all_storms_detailed.head()

,time,extra_obs,special,type,lat,lon,vmax,mslp,wmo_basin,storm_id,year
0,1851-06-25 00:00:00,0,,HU,28.0,-94.8,80,NaN,north_atlantic,AL011851,1851
1,1851-06-25 06:00:00,0,,HU,28.0,-95.4,80,NaN,north_atlantic,AL011851,1851
2,1851-06-25 12:00:00,0,,HU,28.0,-96.0,80,NaN,north_atlantic,AL011851,1851
3,1851-06-25 18:00:00,0,,HU,28.1,-96.5,80,NaN,north_atlantic,AL011851,1851
4,1851-06-25 21:00:00,1,L,HU,28.2,-96.8,80,NaN,north_atlantic,AL011851,1851


In [7]:
# Check to compare previously scraped data
import pandas as pd 
df = pd.read_pickle('data/detailed_storm_data.pkl')
df

,idx,time,extra_obs,special,type,lat,lon,vmax,mslp,wmo_basin,storm_id,storm_name
0,0,1851-06-25 00:00:00,0,,HU,28.0,-94.8,80.0,NaN,north_atlantic,AL011851,UNNAMED
1,1,1851-06-25 06:00:00,0,,HU,28.0,-95.4,80.0,NaN,north_atlantic,AL011851,UNNAMED
2,2,1851-06-25 12:00:00,0,,HU,28.0,-96.0,80.0,NaN,north_atlantic,AL011851,UNNAMED
3,3,1851-06-25 18:00:00,0,,HU,28.1,-96.5,80.0,NaN,north_atlantic,AL011851,UNNAMED
4,4,1851-06-25 21:00:00,1,L,HU,28.2,-96.8,80.0,NaN,north_atlantic,AL011851,UNNAMED
...,...,...,...,...,...,...,...,...,...,...,...,...
55287,55287,2024-10-21 12:00:00,0,,TS,20.3,-75.4,40.0,1000.0,north_atlantic,AL162024,OSCAR
55288,55288,2024-10-21 18:00:00,0,,TS,20.7,-75.9,35.0,1003.0,north_atlantic,AL162024,OSCAR
55289,55289,2024-10-22 00:00:00,0,,TS,21.5,-75.6,35.0,1005.0,north_atlantic,AL162024,OSCAR
55290,55290,2024-10-22 06:00:00,0,,TS,22.2,-75.1,35.0,1006.0,north_atlantic,AL162024,OSCAR
